# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.51,83,99,5.98,PN,1718667527
1,1,kitimat,54.0524,-128.6534,22.21,56,40,3.58,CA,1718667527
2,2,rabaul,-4.1967,152.1721,29.24,72,34,7.45,PG,1718667527
3,3,taourirt,34.4073,-2.8973,23.60,35,1,0.50,MA,1718667528
4,4,port-aux-francais,-49.3500,70.2167,3.21,68,32,11.71,TF,1718667528


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_city=city_data_df.hvplot.points("Lng","Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    size = "Humidity",
    color = "City"  
)
# YOUR CODE HERE

# Display the map
map_plot_city
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
results_city=city_data_df[(city_data_df['Humidity'] < 70) & 
                                  (city_data_df['Cloudiness'] == 0) & 
                                  (city_data_df['Max Temp'] <= 27) & 
                                  (city_data_df['Max Temp'] >= 21)]
# YOUR CODE HERE

# Drop any rows with null values
results_city_null_values= results_city.dropna()
# YOUR CODE HERE

# Display sample data
results_city_null_values
# YOUR CODE HERE

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
58,58,lompoc,34.6391,-120.4579,25.99,62,0,9.26,US,1718667538
190,190,barra,-11.0894,-43.1417,23.62,49,0,3.01,BR,1718667564
290,290,al bayda',32.7627,21.7551,21.00,42,0,2.09,LY,1718667581
333,333,westport,41.1415,-73.3579,26.63,66,0,4.12,US,1718667340
346,346,formosa do rio preto,-11.0483,-45.1931,21.06,38,0,1.26,BR,1718667592
449,449,kabalo,-6.0500,26.9167,21.90,45,0,1.84,CD,1718667613
465,465,birjand,32.8663,59.2211,24.26,23,0,4.12,IR,1718667616
546,546,port moresby,-9.4431,147.1797,26.68,61,0,2.57,PG,1718667485


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=results_city[['City', 'Lat', 'Lng', 'Humidity']].copy()
# YOUR CODE HERE 'Country'

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''
# YOUR CODE HERE

# Display sample data
hotel_df
# YOUR CODE HERE

,City,Lat,Lng,Humidity,Hotel Name
58,lompoc,34.6391,-120.4579,62,
190,barra,-11.0894,-43.1417,49,
290,al bayda',32.7627,21.7551,42,
333,westport,41.1415,-73.3579,66,
346,formosa do rio preto,-11.0483,-45.1931,38,
449,kabalo,-6.0500,26.9167,45,
465,birjand,32.8663,59.2211,23,
546,port moresby,-9.4431,147.1797,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 1000# YOUR CODE HERE
params = {"apiKey": geoapify_key,
    "limit":20,
    'categories':'accommodation.hotel'
    # YOUR CODE HERE
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"# YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}"# YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)# YOUR CODE HERE
    
    # Convert the API response to JSON format
    name_address = name_address.json()# YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lompoc - nearest hotel: No hotel found
barra - nearest hotel: No hotel found
al bayda' - nearest hotel: فندق الهديل
westport - nearest hotel: No hotel found
formosa do rio preto - nearest hotel: Pousada Universo - Formosa do Rio Preto BA
kabalo - nearest hotel: No hotel found
birjand - nearest hotel: No hotel found
port moresby - nearest hotel: The Sanctuary Hotel and Spa


,City,Lat,Lng,Humidity,Hotel Name
58,lompoc,34.6391,-120.4579,62,No hotel found
190,barra,-11.0894,-43.1417,49,No hotel found
290,al bayda',32.7627,21.7551,42,فندق الهديل
333,westport,41.1415,-73.3579,66,No hotel found
346,formosa do rio preto,-11.0483,-45.1931,38,Pousada Universo - Formosa do Rio Preto BA
449,kabalo,-6.0500,26.9167,45,No hotel found
465,birjand,32.8663,59.2211,23,No hotel found
546,port moresby,-9.4431,147.1797,61,The Sanctuary Hotel and Spa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display 

# Configure the map plot 
map_plot_city2 = hotel_df.hvplot.points("Lng","Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height =600,
    size = "Humidity",
    color = "City",
    hover_cols = ['City', 'Country', 'Hotel Name'] 
)
# YOUR CODE HERE

# Display the map
map_plot_city2
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)